* https://scottwales.github.io/swc-climatedata/02-xarray/
* https://geohackweek.github.io/nDarrays/02-xarray-architecture/

In [1]:
import xarray as xr
import numpy as np

#%matplotlib inline 

In [2]:
fileName = 'macav2livneh_pr_bcc-csm1-1_r1i1p1_historical_1990_2005_CONUS_monthly.nc'

In [3]:
#Create the XArray dataset object
ds = xr.open_dataset(fileName)

TypeError: Error: C:\Workspace\Gits\environ859\SandBox\wenhong\macav2livneh_pr_bcc-csm1-1_r1i1p1_historical_1990_2005_CONUS_monthly.nc is not a valid NetCDF 3 file
            If this is a NetCDF4 file, you may need to install the
            netcdf4 library, e.g.,

            $ pip install netcdf4
            

In [ ]:
precip = ds['precipitation']

# Single value

In [ ]:
#Label based indexing
val = ds['precipitation'][0,250,300]
val.values

In [ ]:
ds['lat'].values[0]

# Map of precipitation for a single time snapshot

In [ ]:
#Extract the time at t0
t0 = ds['time'].values[0]
t0

In [ ]:
#Subset all locations at time=t0
map_t0 = ds['precipitation'].loc[t0,:,:]
map_t0.plot(figsize=(10,5),cmap="YlGnBu");

# Time averaged map

In [ ]:
#Create a time slice
startTime = np.datetime64('1995')
endTime = np.datetime64('2000')
#
timeX = slice(startTime,endTime)
latX = slice(245,250)
lonX = slice(300,305)
time_series = ds['precipitation'].sel(time=timeX)
m = time_series.mean(axis=0)
m.shape

In [ ]:
time_series.median(axis=0).plot(figsize=(10,5),cmap="YlGnBu");

# Time series: one location

In [ ]:
#http://xarray.pydata.org/en/stable/indexing.html#nearest-neighbor-lookups


In [ ]:
theLats = slice(40,41)
theLons = slice(254.5,255)
selX = ds.sel(lat=theLats,lon=theLons)
selX.precipitation.mean(axis=0).plot(cmap="YlGnBu")

In [ ]:
theLat = 40
theLon= 254
timeSeries = ds.sel(lat=theLat,lon=theLon,method='nearest')
timeSeries.precipitation.plot();

# Summer (JJA) average

In [ ]:
#read variables in NETCDF file
lon = ds.lon.values
lat = ds.lat.values
time = ds.time.values
precip = ds.precipitation.values

In [ ]:
#return the dimension size of the variable precip
[ntime,nlon,nlat] = precip.shape
print(ntime,nlon,nlat)

In [ ]:
#return the attribute of the variable precip
units = ds.precipitation.units
print(units)

In [ ]:
#replace fillvalues with nan(not a number)
##-->not needed here as 'fill values' are already set to 'nan'
precip[0,90:100,300]

In [ ]:
#JJA slice: year = 1990
def getJJA(year = 1990):
    start_time = np.datetime64('{}-06-01'.format(year))
    end_time = np.datetime64('{}-08-31'.format(year))
    data_slice = ds.sel(time=slice(start_time,end_time))
    mean_ppt = data_slice.precipitation.mean(axis=0)
    return mean_ppt

In [ ]:
#calculate summer(JJA) average
nyear = int(ntime/12)
year = np.arange(1990,2005)
precip_JJA = np.zeros((nyear,nlon,nlat))
for ii in range(nyear):
    theYear = 1990+ii
    precip_JJA[ii,:,:] = getJJA(theYear)

In [ ]:
precip_JJA_climate = precip_JJA.mean(axis=0)

# Plot

In [ ]:
#Import the plotting components
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
precip_JJA_climate_std = precip_JJA.std(axis=0) 

In [ ]:
mlon,mlat = np.meshgrid(lon,lat)
#First frame: precipitiation
plt.figure(figsize=(10,10))
plt.subplot(2,1,1)
plt.contourf(mlon,mlat,precip_JJA_climate,
             linestyles='none',
             cmap="YlGnBu",
             levels=np.arange(0,250,5))
plt.colorbar(label='precipitation '+units)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('climatology of the summer precipitation')

#Second frame: std deviation
plt.subplot(2,1,2)
plt.contourf(mlon,mlat,precip_JJA_climate_std,
             cmap="bone",
             linestyles='none',
             levels=np.arange(0,50))
plt.colorbar(label='precipitation '+units)
plt.xlabel('Longitude')
plt.ylabel('Latitude');
plt.title('standard deviation of the summer precipitation')

plt.show();